# Часть 2 
# Оработка данных используя различные библиотеки для анализа данных
## Подготовка к работе и выгрузка данных
### Импорт библиотек

In [16]:
import psycopg2
import pandas as pd
import psycopg2.extras

### Поделючение к базе данных

Откроем подключение к базе данных, а также настроим курсор для взаимодействия с ней в дальнейшем.

In [17]:
conn = psycopg2.connect(dbname='theLook', user='postgres', host='localhost')
cursor = conn.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor)

### Выгрузка данных

Проверим подключение и выгрузим данные о 10 пользователях.

In [18]:
cursor.execute('SELECT * FROM users')
all_users = cursor.fetchmany(size=10)
df = pd.DataFrame(all_users)
df

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2022-07-19 13:51:00+00:00
1,6578,Elizabeth,Martinez,elizabethmartinez@example.com,34,F,Acre,1705 Nielsen Land,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2023-11-08 18:49:00+00:00
2,36280,Christopher,Mendoza,christophermendoza@example.net,13,M,Acre,125 Turner Isle Apt. 264,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Email,2019-08-24 06:10:00+00:00
3,60193,Jimmy,Conner,jimmyconner@example.com,64,M,Acre,0966 Jose Branch Apt. 008,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-02-15 11:26:00+00:00
4,64231,Natasha,Wilson,natashawilson@example.net,25,F,Acre,20798 Phillip Trail Apt. 392,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-03-13 06:45:00+00:00
5,72187,Andrea,Bryant,andreabryant@example.org,47,F,Acre,622 Sims Field,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2022-05-27 14:53:00+00:00
6,22528,Keith,Barnett,keithbarnett@example.com,64,M,Acre,53908 Amy Fork Apt. 281,69940-000,Sena Madureira,Brasil,-9.857325,-69.43706,Search,2019-03-21 10:58:00+00:00
7,34417,Joanne,Mcpherson,joannemcpherson@example.com,28,F,Acre,53907 John Unions Apt. 099,69940-000,Sena Madureira,Brasil,-9.857325,-69.43706,Search,2024-01-12 19:02:31.096664+00:00
8,50098,Ian,Burke,ianburke@example.net,52,M,Acre,91902 Andrea Stream Apt. 563,69940-000,Sena Madureira,Brasil,-9.857325,-69.43706,Email,2019-04-12 06:11:00+00:00
9,63352,David,Orozco,davidorozco@example.org,18,M,Acre,9576 Dominique Run,69940-000,Sena Madureira,Brasil,-9.857325,-69.43706,Search,2023-10-29 12:33:00+00:00


## Цели 
### 1) Когортный анализ удержания (Retention)
    Создать когорты пользователей по месяцу их первого заказа. Рассчитайть Retention Rate для каждой когорты на протяжении нескольких месяцев.
### 2) Сегментация клиентов (RFM-анализ)
    Разделить всех клиентов на сегменты на основе трех метрик:
* Recency (Давность) — как давно клиент совершил последний заказ.
* Frequency (Частота) — как часто он совершает заказы.
* Monetary (Денежная ценность) — общая сумма его заказов.
    Присвоить каждому клиенту RFM-скор (например, от 1 до 5 по каждой метрике) и на основе этого выделите сегменты клиентов.
### 3) Анализ товаров
    Определить самые продаваемые товары и категории. Проанализировать, какие товары чаще всего покупают вместе.

## Когоротный анализ удержания.

### Выгрузка данных о пользователях.
Выгрузим данные о пользователях из таблицы users, переведем в dataframe.

In [55]:
cursor.execute('SELECT * FROM users')
all_users = cursor.fetchall()
df = pd.DataFrame(all_users)
display(df.head())
df.describe()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2022-07-19 13:51:00+00:00
1,6578,Elizabeth,Martinez,elizabethmartinez@example.com,34,F,Acre,1705 Nielsen Land,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2023-11-08 18:49:00+00:00
2,36280,Christopher,Mendoza,christophermendoza@example.net,13,M,Acre,125 Turner Isle Apt. 264,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Email,2019-08-24 06:10:00+00:00
3,60193,Jimmy,Conner,jimmyconner@example.com,64,M,Acre,0966 Jose Branch Apt. 008,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-02-15 11:26:00+00:00
4,64231,Natasha,Wilson,natashawilson@example.net,25,F,Acre,20798 Phillip Trail Apt. 392,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-03-13 06:45:00+00:00


,id,age,latitude,longitude
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,50000.500000,41.054790,28.315885,25.020268
std,28867.657797,17.043914,22.024165,89.989960
min,1.000000,12.000000,-43.007538,-158.164930
25%,25000.750000,26.000000,26.148664,-50.794605
50%,50000.500000,41.000000,35.219885,4.793601
75%,75000.250000,56.000000,40.707770,116.376110
max,100000.000000,70.000000,64.865200,153.560240


### Объеденяем в кагорты

Создадим кагорты по месяцам регистрации пользователей.

Преобразование дат в единый вид, пригодный для дальнейшей обработки.

In [56]:
df['created_at'] = pd.to_datetime(df['created_at'].str[:19])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              100000 non-null  int64         
 1   first_name      100000 non-null  object        
 2   last_name       100000 non-null  object        
 3   email           100000 non-null  object        
 4   age             100000 non-null  int64         
 5   gender          100000 non-null  object        
 6   state           100000 non-null  object        
 7   street_address  100000 non-null  object        
 8   postal_code     100000 non-null  object        
 9   city            100000 non-null  object        
 10  country         100000 non-null  object        
 11  latitude        100000 non-null  float64       
 12  longitude       100000 non-null  float64       
 13  traffic_source  100000 non-null  object        
 14  created_at      100000 non-null  date

Присваиваем каждому пользователю группу какгорты в зависимости от месяца регестрации.

In [66]:
df['cohort_group'] = df['created_at'].dt.strftime('%Y-%m')
df.groupby('cohort_group').agg({'id':'count','email':'nunique'}).reset_index().rename(columns={'id': 'user_count'})

,cohort_group,user_count,email
0,2019-01,1591,1587
1,2019-02,1441,1435
2,2019-03,1639,1635
3,2019-04,1608,1603
4,2019-05,1727,1716
...,...,...,...
56,2023-09,1620,1612
57,2023-10,1644,1640
58,2023-11,1576,1568
59,2023-12,1635,1627


    Анализ
Можно заметить, что в среднем на сайт приходят примерно 1400-1700 пользователей в месяц. Не все из них уникальные, возможно в магазине есть лазейка с бонусами, при которой лучше создавать новый аккаунт, чем содержать старый. Также можно заметить, что в январе 2024 года произошел резкий прирост пользователей, что может говорить о том, что была проведена хорошая pr компания, наработки которой можно использовать в будущем.

### Сбор информации
Надо определить сколько человек из когорты за каждый месяц делали покупки и на какую сумму в общем.

Для начала достанем таблицу с заказами и для каждого определим для каждого пользователя его когорту. 

In [ ]:
cursor.execute('SELECT * FROM order_items')
all_orders = cursor.fetchall()
orders_df = pd.DataFrame(all_orders)
orders_df = orders_df.merge(df[['id','cohort_group']], left_on='user_id', right_on='id', how='left')
orders_df.sort_values(by='created_at', inplace=True)
orders_df.reset_index(drop=True, inplace=True)

,id_x,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price,id_y,cohort_group
0,43359,29835,23839,1986,116945,Shipped,2019-01-06,2019-01-09 04:36:00+00:00,,,32.00,23839,2019-01
1,43361,29835,23839,12305,116952,Shipped,2019-01-06,2019-01-09 04:36:00+00:00,,,15.00,23839,2019-01
2,43360,29835,23839,1887,116949,Shipped,2019-01-07,2019-01-09 04:36:00+00:00,,,80.00,23839,2019-01
3,167395,115297,92102,4865,451930,Cancelled,2019-01-07,,,,44.50,92102,2019-01
4,123449,85009,67807,20973,333223,Processing,2019-01-11,,,,319.00,67807,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181754,56209,38688,30979,9822,151679,Processing,2024-01-21,,,,79.00,30979,2024-01
181755,17723,12256,9831,12110,47852,Shipped,2024-01-21,2024-01-18 16:12:00+00:00,,,38.00,9831,2024-01
181756,106123,73103,58372,8778,286365,Complete,2024-01-21,2024-01-17 15:53:00+00:00,2024-01-21 15:47:00+00:00,,699.00,58372,2024-01
181757,169764,116925,93394,15687,458334,Cancelled,2024-01-21,,,,32.99,93394,2024-01


Создадим два датафрейма, столбцы которых будут месяца, строчки когорты, значения кол-во активных пользователей и сумма покупок за месяц.

In [87]:
orders_df['created_at'] = pd.to_datetime(orders_df['created_at'])
orders_df['order_month'] = orders_df['created_at'].dt.strftime('%Y-%m')

active_users = orders_df.groupby(['cohort_group', 'order_month'])['user_id'].nunique().unstack(fill_value=0)

total_sum = orders_df.groupby(['cohort_group', 'order_month'])['sale_price'].sum().unstack(fill_value=0)

print("Количество активных пользователей:")
display(active_users)
print("Сумма покупок:")
display(total_sum)

Количество активных пользователей:


order_month,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,...,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
cohort_group,,,,,,,,,,,,,,,,,,,,,
2019-01,22,24,34,33,27,39,41,27,27,40,...,37,25,28,32,35,25,38,33,32,25
2019-02,0,13,30,24,42,35,25,27,30,29,...,24,32,36,32,31,33,34,32,30,14
2019-03,0,0,23,36,39,34,37,30,40,32,...,34,47,31,39,46,34,29,37,35,17
2019-04,0,0,0,8,47,32,30,45,35,28,...,31,50,43,37,41,35,37,35,33,18
2019-05,0,0,0,0,22,38,36,35,48,29,...,32,43,34,35,47,32,52,36,32,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,218,445,463,444,255
2023-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,289,571,576,355
2023-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,405,774,451


Сумма покупок:


order_month,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,...,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
cohort_group,,,,,,,,,,,,,,,,,,,,,
2019-01,1965.08,2251.59,4134.93,3628.37,1983.50,2936.42,3778.47,2637.28,2153.62,2502.82,...,3059.96,2111.32,2339.44,4081.88,3718.18,1892.34,3473.58,2860.21,2546.14,1478.63
2019-02,0.00,1764.30,1766.45,1656.12,3497.43,2709.09,2099.83,2523.97,1997.77,2646.56,...,1913.65,2237.30,3661.80,2910.31,4234.01,3684.54,3006.84,2317.08,3087.22,2300.22
2019-03,0.00,0.00,2186.96,3100.03,4015.29,3061.07,2938.41,2596.69,4253.62,2930.46,...,2998.35,4556.18,3201.55,3600.01,3673.06,3733.03,3007.49,3372.13,2375.28,1328.45
2019-04,0.00,0.00,0.00,719.93,4933.99,3432.17,1933.58,3534.67,2252.15,2296.73,...,2632.31,4880.89,2321.11,4007.10,3004.51,2592.29,3060.96,2251.85,2076.91,2306.25
2019-05,0.00,0.00,0.00,0.00,1698.22,2748.50,3136.65,2770.62,3954.24,1992.37,...,2948.02,3243.22,2737.77,2400.63,5543.49,2555.10,4219.16,3059.97,2525.55,2613.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,19752.06,46057.63,43419.20,45725.74,23914.48
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,28820.18,55301.55,58881.83,37069.08
2023-11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36040.75,84369.63,46734.81


Вычислим для каждой когорты retantion rate(кол-во месяцев, где удержание > 50%), а также значение заработных денег на человека за месяц.

## Закрытие соединения

In [88]:
cursor.close()
conn.close()